In [53]:
import pandas as pd
import numpy as np

Biomas Municipais:

In [54]:
biomas_municipais_raw = pd.read_excel(r"..\Dados\MAPBIOMAS\tabela_geral_mapbiomas_col8_biomas_municipios.xlsx", sheet_name="COBERTURA_COL8.0")

In [55]:
biomas_municipais = biomas_municipais_raw.copy()
biomas_municipais.head()

,feature_id,biome,municipality,state_acronym,geocode,biome [municipality],class_id,level_0,level_1,level_2,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,58776,Amazônia,Alta Floresta D'Oeste,RO,1100015,Amazônia [Alta Floresta D'Oeste],3,Natural,1. Forest,Forest Formation,...,398215.453454,397840.584106,395723.954822,393172.594616,390143.107808,386524.612944,381626.249349,376914.362278,371808.971918,363346.377392
1,58776,Amazônia,Alta Floresta D'Oeste,RO,1100015,Amazônia [Alta Floresta D'Oeste],4,Natural,1. Forest,Savanna Formation,...,18046.214376,18074.921873,18098.535228,18076.093824,18129.441959,18192.191212,18301.638850,18149.860775,17479.215459,17245.182795
2,58776,Amazônia,Alta Floresta D'Oeste,RO,1100015,Amazônia [Alta Floresta D'Oeste],6,Natural,1. Forest,Flooded Forest,...,1275.530057,1272.826258,1266.101782,1261.999820,1256.767919,1250.220658,1235.557508,1225.253080,1185.364271,875.174968
3,58776,Amazônia,Alta Floresta D'Oeste,RO,1100015,Amazônia [Alta Floresta D'Oeste],11,Natural,2. Non Forest Natural Formation,Wetland,...,60175.475176,59993.441257,59219.810887,58316.209550,57184.780407,54729.334858,54080.286962,52074.295988,53881.348039,55109.822446
4,58776,Amazônia,Alta Floresta D'Oeste,RO,1100015,Amazônia [Alta Floresta D'Oeste],12,Natural,2. Non Forest Natural Formation,Grassland,...,29190.593141,28929.128789,30427.775226,31215.734618,32524.642042,34829.659165,35405.621437,39093.220041,35638.957528,26916.237147


In [56]:
colunas_esqcolhidas = ["geocode", "class_id", "level_0", "level_1", "level_2", "level_3", "level_4", 2022]
biomas_municipais = biomas_municipais[colunas_esqcolhidas]
biomas_municipais.rename(columns={"geocode":"IBGE7", 2022:"AREA_2022"}, inplace=True)

In [57]:
biomas_municipais["level_2"].unique()

array(['Forest Formation', 'Savanna Formation', 'Flooded Forest',
       'Wetland', 'Grassland', 'Rocky outcrop', 'Pasture', 'Agriculture',
       'Urban Infrastructure', 'River, Lake and Ocean', 'Non Observed',
       'Mining', 'Forest Plantation', 'Mosaic of Agriculture and Pasture',
       'Other Non Vegetated Area', 'Salt flat', 'Magrove',
       'Beach and Dune', 'Aquaculture', 'Restinga Herbácea/Arbustiva',
       'Other Non Forest Natural Formation', 'Wooded Restinga'],
      dtype=object)

In [58]:
biomas_agrupados = biomas_municipais.groupby(["IBGE7", "level_2"])[["AREA_2022"]].sum().reset_index()
biomas_agrupados[biomas_agrupados["IBGE7"] == 2916500]

,IBGE7,level_2,AREA_2022
20550,2916500,Agriculture,7694.679164
20551,2916500,Forest Formation,1411.868114
20552,2916500,Forest Plantation,1079.648681
20553,2916500,Grassland,52.910017
20554,2916500,Mosaic of Agriculture and Pasture,10074.272161
20555,2916500,Other Non Vegetated Area,221.456788
20556,2916500,Pasture,93222.898002
20557,2916500,"River, Lake and Ocean",53.148865
20558,2916500,Savanna Formation,41684.395434
20559,2916500,Urban Infrastructure,267.814283


Superfície de água

In [43]:
superficie_agua = pd.read_excel(r"..\Dados\MAPBIOMAS\Estatisticas_SuperficieÁgua_Col2_SITE.xlsx", sheet_name="mun_annual")

In [44]:
superficie_agua.rename(columns={"code":"IBGE7", "area_ha":"AREA_AGUA", "year":"ANO", "state":"UF"}, inplace=True)
superficie_agua = superficie_agua[superficie_agua["ANO"] >= 2000]

In [45]:
superficie_agua

,IBGE7,city,UF,ANO,AREA_AGUA
83580,1100015,Alta Floresta D'Oeste,RO,2000,6915.636161
83581,1100023,Ariquemes,RO,2000,1553.813914
83582,1100031,Cabixi,RO,2000,437.609037
83583,1100049,Cacoal,RO,2000,1000.792724
83584,1100056,Cerejeiras,RO,2000,702.045298
...,...,...,...,...,...
211731,5222005,Vianópolis,GO,2022,145.143010
211732,5222054,Vicentinópolis,GO,2022,542.161117
211733,5222203,Vila Boa,GO,2022,271.537593
211734,5222302,Vila Propício,GO,2022,796.216708
